# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

# Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

# Part 2 - Create ETL pipeline for the business data from the API

## Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

## Request

### Imports and Setup

In [1]:
import requests
import pandas as pd
import json
import csv
import numpy as np

with open(r'C:\Users\jax\.secret\creds.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bmcca\\.secret\\yelp_api.json'

### ƒ: yelp_request

 - Params: search term (eg. "wineries); location; yelp_key variable (from Imports); and changing setting to print details

In [ ]:
def yelp_request(term, location, yelp_key, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

### Sending the request and saving the response

- Un-comment the next line to run the response

In [ ]:
response = yelp_request('winery','Southern California', yelp_key)
response.keys()

#### Saving/Loading as JSON for simplicity while iterating

In [ ]:
# try:
#     with open(r'data\response.txt', 'w') as f:
#         json.dump(response, f)
# except IOError:
#     print("I/O error")

In [ ]:
# with open('data/response.txt') as json_file:
#     data = json.load(json_file)

### Identifying and Exploring Keys

In [ ]:
# Identify keys

print(response.keys())

#### Exploring the "Businesses" Key

In [ ]:
response['businesses']

In [ ]:
# Show first item w/in list of businesses

response['businesses'][0]

In [ ]:
response['businesses'][0]['categories'][0]['alias']

In [ ]:
response['businesses'][0]['categories'][0]['title']

#### Exploring the "Total" Key

In [ ]:
response['total']

# How many businesses are there in total for my request?

#### Exploring the "Region" Key

In [ ]:
response['region']

# From which geographical area will my results come?

## Parse

In [ ]:
response.keys()

### ƒ: parse_data

In [ ]:
def parse_data(list_of_data):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_data = []
    
    # Loop through each business in the list of businesses
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
    for business in list_of_data:
        if 'price' not in business:
            business['price'] = np.nan
            
            # Verify that the "price" key is in the selected business dict
            
        details = {'name': business['name'],
                     'location': ' '.join(business['location']['display_address']),
                     'id': business['id'],
                     #'categories': business['categories'],
                     'alias': business['categories'][0]['alias'],
                     'title': business['categories'][0]['title'],
                     'rating': business['rating'],
                     'review_count': business['review_count'],
                     'price': business['price'],
                     'latitude': business['coordinates']['latitude'],
                     'longitude': business['coordinates']['longitude']
                    }

        # If the "price" key is missing, then skip adding that key
        # This avoids an error when getting the desired info
        
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
        
        # Add the new dictionary to the previous list
        
        parsed_data.append(details)
    
    # Adjust the 'location' value to be a single string (not 2 str in a list)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
    # Create a DataFrame from the resulting list
    
    df_parsed_data = pd.DataFrame(parsed_data)
    
#     df_parsed_data.dropna(inplace=True)
    
    return df_parsed_data

In [ ]:
parsed_results = parse_data(response['businesses'])
parsed_results

In [ ]:
# Identify totals of NaN values

parsed_results.isnull().sum()

In [ ]:
# Simple statistical exploration of quantitative values

parsed_results.describe()

## Updating Requests for Pagination

### ƒ: yelp_request_offset

In [ ]:
def yelp_request_offset(term, location, yelp_key, offset=0, verbose=False):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

#### Test 1

In [ ]:
test1 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)
test1

In [ ]:
test1.keys()

In [ ]:
test1['total']

In [ ]:
len(test1['businesses'])

#### Test 2

In [ ]:
test2 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=50, verbose=False)
test2

#### Loop

In [ ]:
results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

num_pages = results['total']//50+1

print(results['total'])
print(len(results['businesses']))
print(num_pages)

In [ ]:
cur = 50
parsed_results_dfs = []

for num in range(num_pages):
    results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
    parsed_results = parse_data(results['businesses'])
    parsed_results_dfs.append(parsed_results)
    cur += 50
    

In [ ]:
parsed_results_dfs

In [ ]:
df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
df_concat

## Save

### ƒ: df_save

Save parsed results to a .csv file

In [ ]:
# df.shape

In [ ]:
# df_concat.to_csv('data\wineries.csv')

In [ ]:
# df = pd.read_csv("data/wineries.csv", index_col=0)
# df

#### Old Stuff

In [ ]:
# from helper_funcs import *

In [ ]:


# # create a variable  to keep track of which result you are in. 
# cur = 0

# #set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
#     #set the offset parameter to be where you currently are in the results 

#     #make your API call with the new offset number
#     results = yelp_call_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
    
#     #after you get your results you can now use your function to parse those results
#     parsed_results = parse_results(results)
    
#     # use your function to insert your parsed results into the db
#     df_save(parsed_results)
#     #increment the counter by 50 to move on to the next results
#     cur += 50

## Edit and Condense

Goal: condense the details down to specific functions (perhaps one function?) to pull, clean, and save data.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

# ƒ: GET FULL DATA

In [ ]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

In [ ]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = 0
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_data(results['businesses'])
            parsed_results_dfs.append(parsed_results)
            cur += 50
        except:
            print(f'Error on page {num}.')
            parsed_results_dfs.to_csv(file_name, mode='a')

    # Concatenate DataFrames and save to .csv
    df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

    try:
        df_concat.to_csv(file_name)
        print(f'Saved to {file_name}.')
    except:
        print(f'Error, did not save.')
        
    return df_concat

In [ ]:
df2 = get_full_data('winery', 'San Diego', yelp_key)
df2

In [ ]:
# print(df2['name'].to_string())

In [ ]:
# df3 = get_full_data('pizza', 'Baltimore', yelp_key)

In [ ]:
# df3

## Cleaning Data

In [ ]:
# df2.isna().sum()

In [ ]:
# df2[df2['price'].isna()]

## Identifying top 3 aliases

In [ ]:
# Create new DF showing only alias and title columns
df2_alias = df2.loc[:,['alias', 'title']]

# Identify top 2 aliases 
df2_alias_count = df2_alias.groupby('alias').count().sort_values(['title'],\
                                                        ascending=False)[:2]
df2_alias_count.reset_index(inplace=True)

print(df2_alias_count)

# display them as a list
aliases_top_3 = df2_alias_count['alias'].tolist()
print(aliases_top_3)

# Note: initially tried top 3, but it returned distributors, not wineries

In [ ]:
# Selecting rows based on condition

df3 = df2[df2['alias'].isin(aliases_top_3)]
df3

In [ ]:
df3.to_csv('data/filtered_aliases.csv')

In [ ]:
df_saved = pd.read_csv("data/filtered_aliases.csv", index_col=0)
df_saved

### Ignore - old code

In [ ]:
# df2_alias_sorted = df2_alias_count.sort_values(['title'], ascending=False)
# df3_alias = pd.DataFrame(df2_alias_sorted)
# df3_alias

# df3_alias.groupby('alias').sum().sort_values(['title'], ascending=False)[:3]

# df_alias_top_3 = df3_alias.loc[:, 'title']#[:3]
# df_alias_top_3

In [ ]:
# df2_alias = df2.loc[:,'alias']

# df2_alias_count = df2.groupby('alias').count()#.sort_values(['alias'], ascending=False)#[:3]
# df2_alias_count.reset_index(inplace=True)

# df2_alias_count

# Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

## Getting Business IDs

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

### Open file and slice ID

1. Open data/wineries.csv
2. Slice out the 'name' and 'id' columns for each row

In [ ]:
df_saved = pd.read_csv("data/filtered_aliases.csv", index_col=0)
df_saved.reset_index(drop=True, inplace=True)
df_saved

In [ ]:
# Slice out the elements from the "name" column

df_saved_name = df_saved['name'].to_list()
df_saved_name

In [ ]:
df_saved_id = df_saved['id'].to_list()
df_saved_id

## Requesting Reviews

- Write a function that takes a business id and makes a call to the API for reviews.


### ƒ: yelp_id_offset

In [ ]:
#Testing how to create URL properly for API request

biz_id = "DknnpiG1p4OoM1maFshzXA"
url = 'https://api.yelp.com/v3/businesses/' + biz_id + '/reviews'
url

In [ ]:
# f'https /{biz_id}/reviews

In [ ]:
# for biz in biz_ids:
#     reviews = reviewcall(biz)
#     parsed_reviews = parse_reviews(reviews, biz)
    

In [ ]:
def yelp_id_offset(biz_id, yelp_key, offset=0, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/'+ f'{biz_id}' + '/reviews'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
#                     'id': biz_id.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
        
    return response.json()

#### Test ID Offset

In [ ]:
# print(len(reviews1['reviews']))

##### Explore Reviews1 Keys

In [ ]:
# reviews_list

In [ ]:
# print(type(reviews_list['reviews']))

In [ ]:
# reviews1['reviews'][0]

In [ ]:
# reviews1['reviews'][0].keys()

In [ ]:
# reviews1['reviews'][0]['user']

### ƒ: Parse_Reviews

In [ ]:
def parse_reviews(list_of_reviews):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_reviews = []
    
    # Loop through each review in the list of reviews
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
#     print('Completed: create list ' + f'{parsed_reviews}')
    
    for review in list_of_reviews:
        details = {'Reviewer Name': review['user']['name'],
            'Review ID': review['id'],
            'Time Created': review['time_created'],
            'Review Rating': review['rating'],
            'Review Text': review['text']
            }
        
        # Add the new dictionary to the empty list
        
        parsed_reviews.append(details)
    
#     print('Completed: append list')
    
    # Create a DataFrame from the resulting list
    
    df_parsed_reviews = pd.DataFrame(parsed_reviews)

#     print('Created DF')
    
    return df_parsed_reviews

In [ ]:
# df_parsed_reviews = parse_reviews(reviews1['reviews'])
# # df_parsed_reviews

### ƒ: Get_Full_Reviews

In [ ]:
def get_reviews(biz_id, yelp_key,verbose=False):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    if verbose == True:
        print(f'For {biz_id}: ')
        print(f"    Total number of results: {results['total']}.")
        print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = len(results['reviews'])
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(1,num_pages):
        try:
            results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_reviews(results['reviews'])
            parsed_results['Business ID'] = biz_id
            parsed_results_dfs.append(parsed_results)
            cur += len(results['reviews'])
        except Exception as e:
            print(f'Error on page {num}.')
            print(e)

    # Concatenate DataFrames and save to .csv
    try:
        df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
        df_concat.to_csv('data/business_ids.csv')
        return df_concat
    except Exception as e:
        print(f'Error, did not concat.')
        print(e)
#         return parsed_results_dfs


#     try:
#         df_concat.to_csv('data/business_ids.csv')
# #         print(f'Saved to data/business_ids.csv.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

In [ ]:
biz_id

In [ ]:
# results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
# num_pages = 2 #results['total']//50+1

# # Print out confirmation feedback
# print(f'For {biz_id}: ')
# print(f"    Total number of results: {results['total']}.")
# print(f'    Total number of pages: {num_pages}.')

# # Create offset for further results and create empty list
# cur = 50
# parsed_results_dfs = []

# # Retrieves remaining pages
# for num in range(num_pages-1):
#     try:
#         results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
#         parsed_results = parse_reviews(results['reviews'])
#         parsed_results_dfs.append(parsed_results)
#         cur += 50
#     except Exception as e:
#         print(f'Error on page {num}.')
#         print(e)

# # Concatenate DataFrames and save to .csv
# df_concat = pd.concat(parsed_results_dfs, ignore_index=True)


In [ ]:
reviews_list = []

for biz_id in df_saved_id:
    review_result = get_reviews(biz_id, yelp_key)
    if review_result is not None:
        reviews_list.append(review_result)

reviews_list
    


In [ ]:
df_reviews = pd.concat(reviews_list, ignore_index=True)
df_reviews

## Parsing Reviews

- Write a function to parse out the relevant information from the reviews

### Adjust prior function

In [ ]:
# def parse_data(list_of_data):
#     '''Adapted from Tyrell's code'''  


#     parsed_data = []
     
#     for business in list_of_data:
#         if 'price' in business:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'price': business['price'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
    
#         parsed_data.append(details)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
#     df_parsed_data = pd.DataFrame(parsed_data)
    
# #     df_parsed_data.dropna(inplace=True)
    
#     return df_parsed_data

## Saving Parsed Data

- Write a function to save the parse data into a csv file containing all of the reviews. 

### Adjust function to save to new file

In [ ]:
# def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
#     '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
#     # Process first request to Yelp API and calculate number of pages 
#     results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
#     num_pages = results['total']//50+1
    
#     # Print out confirmation feedback
#     print(f'For {term} and {location}: ')
#     print(f"    Total number of results: {results['total']}.")
#     print(f'    Total number of pages: {num_pages}.')
    
#     # Create offset for further results and create empty list
#     cur = 50
#     parsed_results_dfs = []

#     # Retrieves remaining pages
#     for num in range(num_pages-1):
#         try:
#             results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
#             parsed_results = parse_data(results['businesses'])
#             parsed_results_dfs.append(parsed_results)
#             cur += 50
#         except:
#             print(f'Error on page {num}.')

#     # Concatenate DataFrames and save to .csv
#     df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

#     try:
#         df_concat.to_csv(file_name)
#         print(f'Saved to {file_name}.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

## Combining Functions

- Combine the functions above into a single script  

# Part 4 -  Using python and pandas, write code to answer the questions below. 

**Reviews**

Which are the 5 most reviewed businesses in your dataset?

What is the highest rating received in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or  4.5?

What percentage of businesses have a rating less than 3?

---

**Pricing**

What percentage of your businesses have a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

---

**Returing Reviews**

Return the text of the reviews for the most reviewed business. 

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 

Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20